In [6]:
# pip install 
# torch
# pylzma
# tokenizers
# tqdm
# transformers

In [7]:
import torch
import torch.nn as nn
from torch.nn import functional as F
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
